In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import numpy as np
import h5py
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

/home/weihang/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import os
os.getcwd()

'/home/weihang/googledrive/Spring18/AC209/Final Project/AC209---final-project/src'

In [3]:
data = h5py.File("../../../../../../dataset/data_sample.hdf5", "r")
print("data keys: "+str(list(data.keys())))

data keys: ['images', 'labels', 'observation_days']


In [8]:
images = data["images"][:]
print("image shape: (sample, x_size, y_size, epoch) = "+str(images.shape))
labels = data["labels"][:]

image shape: (sample, x_size, y_size, epoch) = (72000, 21, 21, 48)


In [9]:
dic = {'Asteroids':0, 'Constant':1, 'EmptyLigh':2, 'M33Cephei':3, 'RRLyrae':4, 'Supernova':5}
labels_digit = [dic[i] for i in labels]

labels_flatten = np.repeat(labels_digit, 48)
enc = OneHotEncoder()
labels_flatten_onehot = enc.fit_transform(labels_flatten.reshape(-1, 1))

In [10]:
labels_flatten_onehot.shape

(3456000, 6)

In [11]:
flatten_data = images.transpose(0, 3, 1, 2).reshape((72000*48, 21, 21, 1))
flatten_data.shape

(3456000, 21, 21, 1)

In [23]:
sample_idx = np.random.choice(range(len(flatten_data)), 100000)
sample_data = flatten_data[sample_idx]
sample_labels = labels_flatten_onehot[sample_idx]
sample_data.shape

(100000, 21, 21, 1)

In [24]:
x_train, x_val, y_train, y_val = train_test_split(sample_data, sample_labels)

In [25]:
x_train.shape, y_train.shape

((75000, 21, 21, 1), (75000, 6))

In [29]:
def init_model():
    model = Sequential()
    model.add(Conv2D(32, (4,4), input_shape=(21,21,1), padding='same', activation='relu', kernel_initializer='uniform'))
    model.add(MaxPooling2D((3,3), strides=(1,1)))
    model.add(Conv2D(64, (3,3), padding='same', activation='relu', kernel_initializer='uniform'))
    model.add(Conv2D(64, (3,3), padding='same', activation='relu', kernel_initializer='uniform'))
    model.add(MaxPooling2D((3,3), strides=(1,1)))
    model.add(Conv2D(32, (3,3), padding='same', activation='relu', kernel_initializer='uniform'))
    model.add(Conv2D(32, (3,3), padding='same', activation='relu', kernel_initializer='uniform'))
    model.add(Conv2D(32, (3,3), padding='same', activation='relu', kernel_initializer='uniform'))
    model.add(MaxPooling2D((3,3), strides=(1,1)))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(6, activation='softmax'))

    return model

In [33]:
model = init_model()
print(model.summary())
adam = keras.optimizers.Adam(lr=0.0001)#, decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 21, 21, 32)        544       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 19, 19, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 19, 19, 64)        18496     
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 19, 19, 64)        36928     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 17, 17, 32)        18464     
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 17, 17, 32)        9248      
__________

In [34]:
batch_size = 128
epochs = 20

model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_val, y_val),
              shuffle=True)

Train on 75000 samples, validate on 25000 samples
Epoch 1/20
75000/75000 [==============================] - 12s 159us/step - loss: 1.7058 - acc: 0.2507 - val_loss: 1.4385 - val_acc: 0.3749
Epoch 2/20
75000/75000 [==============================] - 11s 150us/step - loss: 1.2715 - acc: 0.4071 - val_loss: 1.1192 - val_acc: 0.4429
Epoch 3/20
75000/75000 [==============================] - 11s 148us/step - loss: 1.1777 - acc: 0.4291 - val_loss: 1.0958 - val_acc: 0.4525
Epoch 4/20
75000/75000 [==============================] - 11s 150us/step - loss: 1.1495 - acc: 0.4376 - val_loss: 1.0769 - val_acc: 0.4535
Epoch 5/20
75000/75000 [==============================] - 11s 151us/step - loss: 1.1143 - acc: 0.4480 - val_loss: 1.0589 - val_acc: 0.4627
Epoch 6/20
75000/75000 [==============================] - 11s 151us/step - loss: 1.1110 - acc: 0.4485 - val_loss: 1.1027 - val_acc: 0.4519
Epoch 7/20
75000/75000 [==============================] - 11s 153us/step - loss: 1.0951 - acc: 0.4567 - val_loss: 1.